In [49]:
import glob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from IPython.core.debugger import set_trace
import difflib
import csv
from scipy.stats import entropy

In [17]:
alldata_new.keys()

dict_keys(['doc_length', 'doc_topic', 'docnames', 'term_topic', 'terms', 'term_frequency', 'docs_per_year'])

In [19]:
# load data

alldata_new = pickle.load(open('output/dtm_processed_output.p', 'rb'))
doctitles=alldata_new['docnames']
doc_year=alldata_new['docs_per_year']
doc_ids =[0]+list(np.cumsum(doc_year))

term_topic = alldata_new['term_topic']# term_topic is n_years*n_topics*n_terms
terms = alldata_new['terms']
term_frequency = alldata_new['term_frequency'][1:] # weirdly the first entry is empty
doc_topicyrs = alldata_new['doc_topic']

doc_topic = []
doc_length=[]
for year in range(len(term_topic)):    
    doc_topic.append(alldata_new['doc_topic'][doc_ids[year]:doc_ids[year+1]])# doc_topic is nyear*n_docs given year*n_topics
    doc_length.append(alldata_new['doc_length'][doc_ids[year]:doc_ids[year+1]]) #doc_length is nyear*n_docs given year"""    
# rename topics by the hand-picked names
topic_labels = pickle.load(open('topicnames.p','rb'))
    

In [3]:
def stringdiff(a,b):
    return sum ( a[i] != b[i] for i in range(len(a)) )

In [46]:
def getlist(titles):
    doclist=[]
    titlelist=[]
    titles = [k.lower() for k in titles]
    for doc in doctitles:
        for title in titles:
            matchratio = difflib.SequenceMatcher(None,title,doc).ratio() 
            if matchratio >.7:
                print(doc+'\n'+title)
                doclist.append(doctitles.index(doc))
                titlelist.append(title)
        if set(titlelist)==set(titles):
            break
                
    for t in titles:
        if t not in titlelist:
            print('\ncannot find: '+t)
    return([doclist,titlelist])

# given a list of paper, what are their main topics? for analyzing like a lab or an author
def maintopics(doclist,*topic_labels):
    ntopics=20
    doc_topfreq=np.empty((len(doclist),ntopics))
    for k in range(len(doclist)):
        if isinstance(doclist[0],int):
            doc_topfreq[k]=alldata_new['doc_topic'][doclist[k]]
        elif len(doclist[0])==2: # year then index
            try:
                doc_topfreq[k]=doc_topic[doclist[k][0]][doclist[k][1]]
            except:
                year=doclist[k][0]
                
                print('year%d'%doclist[k][0])
                print(len(doc_topic[year]))
                docdir = 'text_data/volume_{}/'.format(22+year)
                alldocs = glob.glob(docdir+'*.txt')
                print(len(alldocs))
                set_trace()
    doc_topfreq = np.mean(doc_topfreq,axis=0)
    doc_topfreq = doc_topfreq/sum(doc_topfreq)
    
    maintopid = np.argsort(-doc_topfreq) 
    doc_topfreq=doc_topfreq[maintopid]
    if topic_labels:
        maintopics=[topic_labels[0][idx] for idx in maintopid]
    return (maintopics,doc_topfreq)
def lab_summary(titles,label):
    [doclist,titlelist]=getlist(titles)        
    (mtops,meantpfreq)=maintopics(doclist,topic_labels)
    with open('result/lab_topic/'+label+'.txt','w') as f:
        for k in range(len(mtops)):
            f.write(mtops[k]+', freq={}'.format(meantpfreq[k])+'\n')
        f.write('\n papers included:\n')
        for title in titlelist:
            f.write(title)

    with open('result/lab_topic/'+label+'.csv','w') as f:
        csvwriter = csv.writer(f)
        for k in range(len(mtops)):
            csvwriter.writerow([mtops[k],meantpfreq[k]])
    return(doclist,titlelist,mtops,meantpfreq)

In [47]:
label='alex'
titles=['Computationally reproducible experiments','The Attentional Learning Trap and How to Avoid it','Online Experiments using jsPsych, psiTurk, and Amazon Mechanical Turk']
(doclist,titlelist,mtops,meantpfreq)=lab_summary(titles,label)

The Attentional Learning Trap and How to Avoid It
the attentional learning trap and how to avoid it

cannot find: computationally reproducible experiments

cannot find: online experiments using jspsych, psiturk, and amazon mechanical turk


([5329],
 ['the attentional learning trap and how to avoid it'],
 ['Decision making',
  'Categorization',
  'Consciousness and identity',
  'Visual attention',
  'Causal reasoning',
  'Reasoning',
  'Artificial network and Neuroscience',
  'Face and emotion perception',
  'Educational psychology',
  'Probabilistic modeling',
  'Sequential learning',
  'Mathematical Psychology',
  'Text processing and creativity',
  'Memory',
  'Spatial cognition and embodied cognition',
  'Developmental psychology',
  'non-verbal communication',
  'Language: syntax',
  'Knowledge structure',
  'Language: semantics'],
 array([  5.55000388e-01,   2.32257476e-01,   4.59295041e-02,
          4.25928275e-02,   4.25890392e-02,   3.62707537e-02,
          1.92191536e-02,   1.14202215e-02,   1.02137483e-02,
          4.20926924e-03,   2.97619087e-05,   2.97619054e-05,
          2.97619054e-05,   2.97619054e-05,   2.97619054e-05,
          2.97619053e-05,   2.97619053e-05,   2.97619052e-05,
          2.97619052

In [48]:
label='anselm'
titles=['Asking and evaluating natural language questions']
(doclist,titlelist,mtops,meantpfreq)=lab_summary(titles,label)

Asking and evaluating natural language questions
asking and evaluating natural language questions


In [31]:
# find the paper index for given titles
label='gureckis'
titles = ['Information selection in categorization with stimulus uncertainty','Beliefs about sparsity affect casual experimentation',
         'Asking and evaluating natural language questions','The distorting effects of deciding to stop sampling',
         'Desirable difficulties in the development of active inquiry skills',
         'Active control of study leads to improved recognition memory in children',
          'Deep Neural Networks Predict Category Typicality Ratings for Images',
         'the Attentional Learning Trap and How to Avoid it',
         'Are Biases When Making Causal Interventions Related to Biases in Belief Updating',
         'Decisions to intervene on causal systems are adaptively selected',
         'The value of approaching bad things',
         'A preference for the unpredictable over the informative during self-directed learning',
         'Workshop: Online Experiments using jsPsych, psiTurk, and Amazon Mechanical Turk',
         'Informavores: Active information foraging and human cognition',
         'Does the utility of information influence sampling behavior',
         'One piece at a time: Learning complex rules through self-directed sampling',
         'Sparse category labels obstruct generalization of category membership',
         'One-shot lotteries in the park',
         'Does category labeling lead to forgetting',
         "Don't Stop 'Til You Get Enough: Adaptive Information Sampling in a Visuomotor Estimation Task",
         'Grow your own representations: Computational constructivism',
         'Category Learning Through Active Sampling',
         'The Impact of Perceptual Aliasing on Exploration and Learning in a Dynamic Decision Making Task',
         'Is categorial perception really verbally-mediated perception',
         'Active Learning Strategies in a Spatial Concept Learning Game',
         'When Things Get Worse before they Get Better: Regulatory Fit and Average-Reward Learning in a Dynamic Decision-Making Environment',
         'The emergence of Collective Structure through Individual Interactions',
         'Modeling category intuitiveness',
         'The effect of the internal structure of categories on perception']            
#lab_summary(titles,label)
[doclist,titlelist]=getlist(titles)        

Modeling Category Intuitiveness
modeling category intuitiveness
The Emergence of Collective Structures Through Individual Interactions
the emergence of collective structure through individual interactions
Active Learning Strategies in a Spatial Concept Learning Game
active learning strategies in a spatial concept learning game
When Things Get Worse before they Get Better Regulatory Fit and Average-Reward Learning in a Dynamic Decision-Making Environment
when things get worse before they get better: regulatory fit and average-reward learning in a dynamic decision-making environment
Is categorical perception really verbally mediated perception
is categorial perception really verbally-mediated perception
The Impact of Perceptual Aliasing on Exploration and Learning in a Dynamic Decision Making Task
the impact of perceptual aliasing on exploration and learning in a dynamic decision making task
Category Learning Through Active Sampling
category learning through active sampling
Grow your own

In [40]:
(mtops,meantpfreq)=maintopics(doclist,topic_labels)
with open('result/lab_topic/'+label+'.txt','w') as f:
    for k in range(len(mtops)):
        f.write(mtops[k]+', freq={}'.format(meantpfreq[k])+'\n')
    f.write('\n papers included:\n')
    for title in titlelist:
        f.write(title)

with open('result/lab_topic/'+label+'.csv','w') as f:
    csvwriter = csv.writer(f)
    for k in range(len(mtops)):
        csvwriter.writerow([mtops[k],meantpfreq[k]])

In [41]:
label='m_frank'
titles=['Exploring word learning in a high-density longitudinal corpus',
       'Ad-hoc scalar implicature in adults and children',
       'The structure of the lexicon reflects principles of communication',
       'Effects of language on color discriminability',
       'Informative communication in word production and word learning',
       'Relating activity contexts to early word learning in dense longitudinal data',
       'The learnability of constructed languages reflects typological patterns',
       'The development of predictive processes in children’s discourse understanding',
       'Developmental and postural changes in children’s visual access to faces',
       'Cross-situational word learning respects mutual exclusivity',
       "Contributions of prosodic and distributional features of caregivers' speech in early word ",
       "Language as a cognitive technology: English-speakers match like pirahã when you don't let them count",
       'Continuity of discourse provides information for word learning',
       'How do presentation and context influence representation for functional fixedness tasks?',
       'A pragmatic account of the processing of negative sentences',
       'Beyond transitional probabilities: Human learners impose a parsimony bias in statistical word segmentation',
       'Discovering the signatures of joint attention in child-caregiver interaction',
       'Learning to reason pragmatically with cognitive limitations',
       'Talking with tact: Polite language as a balance between kindness and informativity',
       'Linguistic input is tuned to children’s developmental level',
       'An integrated model of concept learning and word-concept mapping',
       'Markers of discourse structure in child-directed speech',
       'Modeling disambiguation in word learning via multiple probabilistic constraints',
       "Measuring children's visual access to social information using face detection",
       "Principles of generalization for learning sequential structure in language"]
lab_summary(titles,label)

Language as a Cognitive Technology English-Speakers Match Like Pirah When You Dont Let Them Count
language as a cognitive technology: english-speakers match like pirahã when you don't let them count
Principles of Generalization for Learning Sequential Structure in Language
principles of generalization for learning sequential structure in language
Exploring Word Learning in a High-Density Longitudinal Corpus
exploring word learning in a high-density longitudinal corpus
Continuity of Discourse Provides Information for Word Learning
continuity of discourse provides information for word learning
Cross-situational Word Learning Respects Mutual Exclusivity
cross-situational word learning respects mutual exclusivity
Informative Communication in Word Production and Word Learning
informative communication in word production and word learning
Contributions of Prosodic and Distributional Features of Caregivers Speech in Early Word Learning
contributions of prosodic and distributional features of 

In [43]:
label='Murphy'
titles=['A knowledge resonance (KRES) model of category learning','Eyetracking as an implicit measure of category-based induction']
(doclist,titlelist,mtops,meantpfreq)=lab_summary(titles,label)


A Knowledge-Resonance (KRES) Model of Category Learning
a knowledge resonance (kres) model of category learning
Eyetracking as an Implicit Measure of Category-Based Induction
eyetracking as an implicit measure of category-based induction
